In [1]:
import pandas as pd
import  constants
from utils import get_season, remove_duplicates

In [2]:
deodorants_df_0 = pd.read_csv("../clean_data/deodorants_0.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_1 = pd.read_csv("../clean_data/deodorants_1.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_2 = pd.read_csv("../clean_data/deodorants_2.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_3 = pd.read_csv("../clean_data/deodorants_3.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_4 = pd.read_csv("../clean_data/deodorants_4.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_5 = pd.read_csv("../clean_data/deodorants_5.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_6 = pd.read_csv("../clean_data/deodorants_6.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_7 = pd.read_csv("../clean_data/deodorants_7.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_8 = pd.read_csv("../clean_data/deodorants_8.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_9 = pd.read_csv("../clean_data/deodorants_9.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_10 = pd.read_csv("../clean_data/deodorants_10.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])
deodorants_df_11 = pd.read_csv("../clean_data/deodorants_11.csv", parse_dates=['fecha_trans', 'fecha_desde', 'fecha_hasta'])

In [102]:
deodorants_df = pd.concat([deodorants_df_0, deodorants_df_1,deodorants_df_2, deodorants_df_3,
                           deodorants_df_4, deodorants_df_5, deodorants_df_6,
                           deodorants_df_7, deodorants_df_8, deodorants_df_9,
                           deodorants_df_10, deodorants_df_11], axis=0, ignore_index=True)

In [103]:
deodorants_df.duplicated().sum()

0

In [104]:
deodorants_df.shape

(8356436, 26)

In [105]:
deodorants_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8356436 entries, 0 to 8356435
Data columns (total 26 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   idb                     int64         
 1   id_producto             int64         
 2   fecha_trans             datetime64[ns]
 3   fecha_desde             datetime64[ns]
 4   fecha_hasta             datetime64[ns]
 5   stock_unidades          int64         
 6   venta_unidades          int64         
 7   precio_unitario         float64       
 8   sigue_prod_en_rol       int64         
 9   cd_abast                int64         
 10  mov_month               int64         
 11  mov_year                int64         
 12  mov_weekday             int64         
 13  ventas_estimadas_promo  float64       
 14  nro_prom                float64       
 15  duracion_promo          float64       
 16  id_cadena               int64         
 17  desc_cadena             object        
 18  de

In [106]:
deodorants_df['dia_semana'] = deodorants_df['fecha_trans'].dt.day_name()

In [107]:
# Add a column indicating if the day is a weekend day or a weekday
deodorants_df['is_weekend'] = deodorants_df['dia_semana'].apply(lambda x: 1 if x in ['Sunday', 'Saturday'] else 0)

In [108]:
# create new column with 1 or 0 based on conditions
deodorants_df['tiene_promo'] = ((deodorants_df['nro_prom'].notnull()) & (deodorants_df['fecha_desde'].notnull()) & (deodorants_df['fecha_hasta'].notnull())).astype(int)

In [109]:
deodorants_df.drop(['fecha_desde', 'fecha_hasta'], axis=1, inplace=True)

In [110]:
# check for null or NaN values in each column
null_cols = deodorants_df.isnull().any()

# print columns with null or NaN values
print(null_cols[null_cols == True].index.tolist())

['ventas_estimadas_promo', 'nro_prom', 'duracion_promo', 'desc_producto', 'razon_social', 'id_region']


In [112]:
deodorants_df.fillna(value={'ventas_estimadas_promo': constants.DEFAULT_FLOAT,
                            'nro_prom': constants.DEFAULT_FLOAT,
                            'duracion_promo': constants.DEFAULT_FLOAT,
                            'razon_social': constants.DEFAULT_STRING,
                            'id_region' : constants.DEFAULT_FLOAT},
                            inplace=True)

In [113]:
deodorants_df[['ventas_estimadas_promo','nro_prom', 'duracion_promo', 'id_region']] = deodorants_df[['ventas_estimadas_promo','nro_prom', 'duracion_promo', 'id_region']].astype(int)

In [114]:
filtered_df = deodorants_df[~deodorants_df['desc_region'].str.contains('buenos aires')]
filtered_df.size

0

In [115]:
deodorants_df.drop('desc_region', axis=1, inplace=True)

In [116]:
 # Map the month to the corresponding season
deodorants_df['estacion'] = deodorants_df['fecha_trans'].apply(get_season)

/Users/eo/src/deodorants-project/data_cleaning/utils.py:103: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if start_date <= pd.Timestamp(date) <= end_date:


In [121]:
deodorants_df['desc_producto'] = deodorants_df['desc_producto'].astype(str)

In [122]:
deodorants_df['desc_producto'] = deodorants_df['desc_producto'].str.strip()

In [123]:
def remove_duplicates_here(s):
    if s == 'nan':
        return s
    seen = set()
    result = []
    for word in s.split():
        if word not in seen or word == 'nan':
            result.append(word)
            seen.add(word)
    return " ".join(result)

In [124]:
deodorants_df['desc_producto'] = deodorants_df['desc_producto'].apply(remove_duplicates_here)

In [125]:
# check for null or NaN values in each column
null_cols = deodorants_df.isnull().any()

# print columns with null or NaN values
print(null_cols[null_cols == True].index.tolist())

[]


In [126]:
product_grouped_bottom = deodorants_df.groupby([
    'id_producto', 'desc_producto'
])['venta_unidades'].sum().sort_values(ascending=False).reset_index()
product_grouped_bottom

,id_producto,desc_producto,venta_unidades
0,342686,glade desodorante de ambientes lavander nueva ...,109531
1,240911,poett ambiental lavanda nuevo d 0.377 kg,100861
2,429883,glade desodorante de ambientes ocean oasis nue...,89345
3,249486,glade toque rep lavandax2 9gr/6 lavanda 9 grm,86737
4,342745,glade desodorante de ambientes jazmin nueva fo...,72907
...,...,...,...
87,511324,pure aerosol air wick cherry blossom 250.00 mlt,2927
88,466069,glade toque aromatizante de ambientes campos l...,2812
89,284013,nan,2562
90,511325,nan,1983


In [127]:
value = deodorants_df.groupby('id_producto')['venta_unidades'].sum().sort_values(
    ascending=True).reset_index()
value

,id_producto,venta_unidades
0,497936,0
1,511325,1983
2,284013,2562
3,466069,2812
4,511324,2927
...,...,...
87,342745,72907
88,249486,86737
89,429883,89345
90,240911,100861


In [129]:
# Convert estacion and desc_cadena in categorical
deodorants_df['estacion'] = deodorants_df['estacion'].astype('category')
deodorants_df['desc_cadena'] = deodorants_df['desc_cadena'].astype('category')

In [130]:
deodorants_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8356436 entries, 0 to 8356435
Data columns (total 27 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   idb                     int64         
 1   id_producto             int64         
 2   fecha_trans             datetime64[ns]
 3   stock_unidades          int64         
 4   venta_unidades          int64         
 5   precio_unitario         float64       
 6   sigue_prod_en_rol       int64         
 7   cd_abast                int64         
 8   mov_month               int64         
 9   mov_year                int64         
 10  mov_weekday             int64         
 11  ventas_estimadas_promo  int64         
 12  nro_prom                int64         
 13  duracion_promo          int64         
 14  id_cadena               int64         
 15  desc_cadena             category      
 16  longitud                float64       
 17  latitud                 float64       
 18  lo

In [131]:
deodorants_df.to_csv('../clean_data/deodorants_final_merged.csv', index=False)